# Mixed Logit Estimation Exercise

This problem set originates from Professor Ken Train's course on discrete choice.
Many thanks to him for allowing us to adapt it for use with Larch. It is provided
here with only minor modifications.

<a href="https://colab.research.google.com/github/driftlesslabs/larch/blob/main/exercises/mixed-logit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you run this notebook in CoLab, you'll need to install `larch` like this:

In [2]:
!pip install -q larch6

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.3/237.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 3.5 MB/s eta 0:00:00


We will estimate mixed logit models in this problem set. The data represent
consumers' choices among vehicles in stated preference experiments. The data are
from a study that he did for Toyota and GM to assist them in their analysis of
the potential marketability of electric and hybrid vehicles, back before hybrids
were introduced to the public.   

## Vehicle Type Choice Data

In each choice experiment, the respondent was presented with three vehicles, with
the price and other attributes of each vehicle described. The respondent was asked
to state which of the three vehicles he/she would buy if the these vehicles were
the only ones available in the market. There are 100 respondents in the dataset
for this exercise (the full dataset had 500 respondents). Each respondent was
presented with 15 choice experiments, and most respondents answered all 15. The
attributes of the vehicles were varied over experiments, both for a given
respondent and over respondents. The attributes are: price, operating cost in
dollars per month, engine type (gas, electric, or hybrid), range if electric (in
hundreds of miles between recharging), and the performance level of the vehicle
(high, medium, or low). The performance level was described in terms of top speed
and acceleration, and these descriptions did not vary for each level; for example,
"High" performance was described as having a top speed of 100 mph and 12 seconds
to reach 60 mph, and this description was the same for all "high" performance
vehicles.

In [3]:
import pandas as pd

import larch as lx
from larch import PX

raw_data = pd.read_parquet(lx.example_file("vehicle_choice.parquet"))
data = lx.Dataset.construct.from_idca(raw_data)
data["price_scaled"] = data["price"] / 10000
data["opcost_scaled"] = data["opcost"] / 10
raw_data

/usr/local/lib/python3.10/dist-packages/larch/model/numbamodel.py:32: UserWarning: 

#### larch v6 is experimental, and not feature-complete ####
the first time you import on a new system, this package will
compile optimized binaries for your specific machine,  which
may take a little while, please be patient ...

  warnings.warn(  ## Good news, everyone! ##  )


person_id  chosen  price  opcost  max_range  ev  gas  hybrid  \
case_id alt_id                                                                 
1       1               1       0  46763   47.43        0.0   0    0       1   
        2               1       1  57209   27.43        1.3   1    0       0   
        3               1       0  87960   32.41        1.2   1    0       0   
2       1               1       1  33768    4.89        1.3   1    0       0   
        2               1       0  90336   30.19        0.0   0    0       1   
...                   ...     ...    ...     ...        ...  ..  ...     ...   
1483    2             100       0  28036   14.45        1.6   1    0       0   
        3             100       0  19360   54.76        0.0   0    1       0   
1484    1             100       1  24054   50.57        0.0   0    1       0   
        2             100       0  52795   21.25        0.0   0    0       1   
        3             100       0  60705   25.41        1.4   1    0       0   

                hiperf  medhiperf  
case_id alt_id                     
1       1            0          0  
        2            1          1  
        3            0          1  
2       1            1          1  
        2            0          1  
...                ...        ...  
1483    2            0          0  
        3            1          1  
1484    1            0          0  
        2            0          1  
        3            0          0  

[4452 rows x 10 columns]

## First Things First

It is always best to start with a standard logit model. To save you time, the
script to produce a simple MNL model is provided below.

  - Do the estimated coefficients have the expected signs?
  - What is the estimated willingness to pay for a $1 per month reduction in
    operating cost? Note that price is in tens of thousands of dollars, and
    operating cost is in tens of dollars.
  - The variable "medhiperf" is 1 if the vehicle has either medium or high
    performance and 0 if the vehicle has low performance. The variable
    "hiperf" is 1 for high performance and 0 for medium or low performance.
    So, a vehicle with high performance has a 1 for both of these variables.
    The estimated "utility" from each performance level is therefore 0 for
    low performance, 0.3841 for medium performance, and 0.3841+0.1099=0.4940
    for high performance. Or, stated incrementally, the estimates imply that
    going from low to medium performance increases "utility" by 0.3841, while
    going from medium to high performance increases utility by 0.1099. These
    estimates imply diminishing marginal utility of performance.
  - There are three engine types, with alternative specific constants entered
    for two of them (with the gas engine normalized to zero.) What do the
    estimated constants imply about consumers' preferences for electric and
    hybrid vehicles relative to gas?

In [4]:
simple = lx.Model(data)
simple.utility_ca = (
    PX("price_scaled")
    + PX("opcost_scaled")
    + PX("max_range")
    + PX("ev")
    + PX("hybrid")
    + PX("hiperf")
    + PX("medhiperf")
)
simple.choice_ca_var = "chosen"
simple.maximize_loglike(stderr=True, options={"ftol": 1e-9})

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Larch Model Dashboard                                                                        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ optimization complete                                                                        │
│ Log Likelihood Current =      -1399.193115 Best =      -1399.193115                          │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓ │
│ ┃ Parameter                      ┃ Estimate       ┃ Std. Error     ┃ t-Stat     ┃ Null Val ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩ │
│ │ ev                             │ -1.3941457     │  0.27667809    │ -5.039     │  0       │ │
│ │ hiperf                         │  0.10999294    │  0.083826654   │  1.312     │  0       │ │
│ │ hybrid                         │  0.35535701    │  0.12181664    │  2.917     │  0       │ │
│ │ max_range                      │  0.47840485    │  0.17650166    │  2.71      │  0       │ │
│ │ medhiperf                      │  0.38431996    │  0.085456528   │  4.497     │  0       │ │
│ │ opcost_scaled                  │ -0.12876957    │  0.035309706   │ -3.647     │  0       │ │
│ │ price_scaled                   │ -0.4168321     │  0.033163231   │ -12.57     │  0       │ │
│ └────────────────────────────────┴────────────────┴────────────────┴────────────┴──────────┘ │
└──────────────────────────────────────────────────────────────────────────────────────────────┘

      message: Optimization terminated successfully
      success: True
       status: 0
            x: [-1.394e+00  1.100e-01  3.554e-01  4.784e-01  3.843e-01
                -1.288e-01 -4.168e-01]
          nit: 11
          jac: [ 1.045e-02 -3.469e-02  6.472e-02 -2.991e-02 -4.050e-02
                -1.683e-01  2.522e-01]
         nfev: 42
         njev: 11
      n_cases: 1484
 total_weight: 1484.0
      logloss: 0.9428525035272068
      loglike: -1399.193115234375
       stderr: [ 2.767e-01  8.383e-02  1.218e-01  1.765e-01  8.546e-02
                 3.531e-02  3.316e-02]


1.  *Yes, the coefficients have the expected sign: higher the price and operation cost the utility max range and mid-increase the utility,also the high performance bring the additional positive utility.*

2.  *The Willingness to pay for operating cost reduction:
    Willingness to pay = -(operating cost coef/price coef) * (price units/opcost units)
    Willingness to pay = -(-0.1288/-0.4168) * (10000/10)
    Willingness to pay = $3,090*


3.  *Electric vehicles (ev): -1.3941 (lower utility than gas vehicles);
    Hybrid: 0.3554 (higher utility than gas vehicles)
    Consumers prefer hybrid vehicles over gasoline vehicles
    But have lower acceptance of pure electric vehicles, showing costumers' preference for the electric vehicles.*





## Data Scaling

Operating cost is scaled to be in tens of dollars, and price in tens of thousands.
The optimizer operates most effectively when the diagonal of the hessian has about the
same order of magnitude for all parameters, which can usually be accomplished by scaling
variables such that their coefficients are about the same order of magnitude. To see the  
effect of scaling, remove the scaling of operating cost, such that operating cost enters
as dollars rather than tens of dollars. How many more iterations does the optimizer
take to converge when operating cost is in dollars compared to when operating cost
is in tens of dollars? For standard logit, the difference is run time is immaterial,
since estimation is so quick in any case. But when running mixed logit and other
models that require simulation, the difference can be considerable. It is always
helpful, therefore, to run standard logit models to get the scaling right, since
checking various scales is quick in standard logit, and then to use that scaling
when you turn to mixed logit.

**The iteration to reach the convergence for operating cost enters as dollars would likely require significantly more iterations.**

## A First Mixed Logit Model

Now estimate a mixed model with a fixed price coefficient and normal coefficients
5. for all other attributes. This is probably the most common mixed logit specification.
The code to do so is below.  It uses `lx.mixtures.Normal` to define Normal distributed
parameter mixtures, with the first argument naming an existing parameter to mix, which
will be the mean of the distribution, and the
second giving a new parameter name that will be the (estimated) standard deviation of
the mixture. Run this code and examine the output.

In [4]:
mixed = simple.copy()
for k in ["opcost_scaled", "max_range", "ev", "hybrid", "hiperf", "medhiperf"]:
    mixed.mixtures.append(lx.mixtures.Normal(k, f"s_{k}"))
mixed.n_draws = 200
mixed.seed = 42
mixed.maximize_loglike(stderr=True, options={"ftol": 1e-9})

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Larch Model Dashboard                                                                        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ optimization complete                                                                        │
│ Log Likelihood Current =      -1385.494385 Best =      -1385.494385                          │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓ │
│ ┃ Parameter                      ┃ Estimate       ┃ Std. Error     ┃ t-Stat     ┃ Null Val ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩ │
│ │ ev                             │ -3.0324594     │  0.77985984    │ -3.888     │  0       │ │
│ │ hiperf                         │  0.17625185    │  0.12484163    │  1.412     │  0       │ │
│ │ hybrid                         │  0.54668417    │  0.19362453    │  2.823     │  0       │ │
│ │ max_range                      │  0.88934613    │  0.32414827    │  2.744     │  0       │ │
│ │ medhiperf                      │  0.73265571    │  0.20883472    │  3.508     │  0       │ │
│ │ opcost_scaled                  │ -0.21958793    │  0.071507692   │ -3.071     │  0       │ │
│ │ price_scaled                   │ -0.64789734    │  0.10203676    │ -6.35      │  0       │ │
│ │ s_ev                           │ -2.8056249     │  0.75720936    │ -3.705     │  0       │ │
│ │ s_hiperf                       │ -0.52352062    │  0.87023795    │ -0.6016    │  0       │ │
│ │ s_hybrid                       │  1.1433791     │  0.66024369    │  1.732     │  0       │ │
│ │ s_max_range                    │ -0.12805284    │  0.99805182    │ -0.1283    │  0       │ │
│ │ s_medhiperf                    │  1.574706      │  0.61251038    │  2.571     │  0       │ │
│ │ s_opcost_scaled                │  0.51966278    │  0.1854817     │  2.802     │  0       │ │
│ └────────────────────────────────┴────────────────┴────────────────┴────────────┴──────────┘ │
└──────────────────────────────────────────────────────────────────────────────────────────────┘

      message: Optimization terminated successfully
      success: True
       status: 0
            x: [-3.032e+00  1.763e-01 ...  1.575e+00  5.197e-01]
          nit: 35
          jac: [-8.573e-03 -2.145e-02 ...  2.146e-03  9.458e-03]
         nfev: 75
         njev: 35
      n_cases: 1484
 total_weight: 1484.0
      logloss: 0.9336215530765667
      loglike: -1385.494384765625
       stderr: [ 7.799e-01  1.248e-01 ...  6.125e-01  1.855e-01]

- Each random coefficient is distributed $N(B,W^2)$ where B and W are estimated.
  Eg, the operating cost coefficient is estimated to have a mean of $-0.2149$
  and standard deviation of $0.4668$, such that the variance is $0.4668^2=0.2179$.
  Note, however, that the estimated $W$ can be negative, as occurs for some of
  the coefficients. The negative sign in these cases is simply ignored, and
  the standard deviation is the estimate without the negative sign. Here's
  the reason: the parameter that is being estimated is not actually the standard
  deviation; rather it is $W$ such that $W^2$ is the variance.  $W$ and $-W$
  give the same variance, and hence are equivalent. Also, since the standard
  deviation is defined as the square root of the variance, they give the same
  standard deviation. The advantage of estimating $W$ instead of estimating a
  standard deviation is that the optimization routine does not need to embody
  constraints to keep the parameter positive. Another way to see the issue is
  that a random variable that is $N(B,W^2)$ is created as $B+w*\mu$ where $\mu$
  is standard normal, or equivalently as $B-W*\mu$: both result in a term with
  mean $B$ and standard deviation $W$. An implication of this parameterization
  is that the starting values of $W$ should not be set at 0, but rather at some
  value slightly away from zero. The reason is this: Since $W$ and $-W$ are
  equivalent, the true log likelihood is symmetric around $W=0$ and hence is
  flat at $W=0$. (The simulated log likelihood is not exactly symmetric due
  to simulation noise.) If $W=0$ were used as the starting values, the gradient
  would be zero and the optimization routine would have no guidance on the
  direction to move. (If you want, you can change the starting value for $W$
  to zero, and rerun the model. You'll see that it has a smaller improvement
  in the first iteration and takes more iterations to converge.

## Considering Panel Data

The mixed logit model above allows the preference parameters to vary across observations.
But what we really want is preference parameters that vary across people.
Remember, each person in this study was given 15 different questions to consider.
To make preferences vary only by person, we add a `groupid`:

In [5]:
panel = mixed.copy()
panel.groupid = "person_id"
panel.maximize_loglike(stderr=True, options={"ftol": 1e-9})

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Larch Model Dashboard                                                                        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ optimization complete                                                                        │
│ Log Likelihood Current =      -1330.442871 Best =      -1330.442871                          │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓ │
│ ┃ Parameter                      ┃ Estimate       ┃ Std. Error     ┃ t-Stat     ┃ Null Val ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩ │
│ │ ev                             │ -1.7040241     │  0.34314501    │ -4.966     │  0       │ │
│ │ hiperf                         │  0.09206931    │  0.10426283    │  0.8831    │  0       │ │
│ │ hybrid                         │  0.46831666    │  0.16410087    │  2.854     │  0       │ │
│ │ max_range                      │  0.46279531    │  0.22777636    │  2.032     │  0       │ │
│ │ medhiperf                      │  0.50899615    │  0.11354806    │  4.483     │  0       │ │
│ │ opcost_scaled                  │ -0.13888977    │  0.05287471    │ -2.627     │  0       │ │
│ │ price_scaled                   │ -0.50149379    │  0.039681096   │ -12.64     │  0       │ │
│ │ s_ev                           │ -0.9213478     │  0.22131573    │ -4.163     │  0       │ │
│ │ s_hiperf                       │ -0.41932018    │  0.14359809    │ -2.92      │  0       │ │
│ │ s_hybrid                       │  0.84480162    │  0.13322526    │  6.341     │  0       │ │
│ │ s_max_range                    │  0.71030082    │  0.18695351    │  3.799     │  0       │ │
│ │ s_medhiperf                    │  0.60296855    │  0.14173062    │  4.254     │  0       │ │
│ │ s_opcost_scaled                │ -0.34705379    │  0.049326986   │ -7.036     │  0       │ │
│ └────────────────────────────────┴────────────────┴────────────────┴────────────┴──────────┘ │
└──────────────────────────────────────────────────────────────────────────────────────────────┘

      message: Optimization terminated successfully
      success: True
       status: 0
            x: [-1.704e+00  9.207e-02 ...  6.030e-01 -3.471e-01]
          nit: 20
          jac: [-5.905e-05 -2.011e-03 ...  5.379e-04  6.869e-04]
         nfev: 61
         njev: 20
      n_cases: 1500
 total_weight: 188.0
      logloss: 7.076823782413564
      loglike: -1330.44287109375
       stderr: [ 3.431e-01  1.043e-01 ...  1.417e-01  4.933e-02]

- Is this "panel" model better than the naive version?  Why do you think that is?

- What is the estimated distribution of willingness to pay for a 1 dollar
  reduction in operating cost? An advantage of having a fixed price coefficient
  is that this distribution can be derived fairly easily. (If, in contrast,
  the price coefficient is random, the willingness to pay is the ratio of two
  random terms, which is more difficult to deal with.)

- What share of the population is estimated to dislike reductions in operating
  cost? To like high performance less than medium performance? Are these results
  reasonable?



1.  *The panel model statistically better than the naive model,with a  
    higher LL value and captures the repeated choices by same individuals*
2.  *Willingness to Pay for Operating Cost Reduction:
    With fixed price coefficient (-0.50149) and random operating cost coefficient:
    Mean operation cost coefficient: -0.13889
    Stdard deviation opcost coefficient: -0.34705
    Willingness to Pay Distribution：
    Mean Willingness to Pay Distribution = -(-0.13889/-0.50149) * (10000/10) = $2,769 per dollar reduction
    Std dev Willingness to Pay Distribution = -(-0.34705/-0.50149) * (10000/10) = $6,921 per dollar reduction*


3. *Coefficient follows the N(-0.13889, 0.34705)
   While P(opcost_coef > 0) = P(Z > 0.13889/0.34705) = P(Z > 0.40) ;
   About 34.5% population were prone to not prefer the operation cost reduction which is unreasonable.
   High performance coefficient is less than medium performance coefficient,most people prefer medium to high performance,it would be reasonable since high performance could accompany with the other trade off considerations such as short life time of the vehicles.*






    




## Simulation Noise

Let's explore the effects of simulation noise. The seed is currently set at `42`
(which is, of course, the [correct](https://en.wikipedia.org/wiki/Phrases_from_The_Hitchhiker%27s_Guide_to_the_Galaxy#Answer_to_the_Ultimate_Question_of_Life,_the_Universe,_and_Everything_(42))
value). Change the seed to other values to see the effect of different random
draws on the estimation results. Try three or four seeds, to get a sense of
how much change there is.  Also try changing the number of random draws. With
more random draws, does the effect of differing random seeds change?

In [5]:
mixed = simple.copy()
for k in ["opcost_scaled", "max_range", "ev", "hybrid", "hiperf", "medhiperf"]:
    mixed.mixtures.append(lx.mixtures.Normal(k, f"s_{k}"))
mixed.n_draws = 200
mixed.seed = 100
mixed.maximize_loglike(stderr=True, options={"ftol": 1e-9})

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Larch Model Dashboard                                                                        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ optimization complete                                                                        │
│ Log Likelihood Current =      -1385.564575 Best =      -1385.564575                          │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓ │
│ ┃ Parameter                      ┃ Estimate       ┃ Std. Error     ┃ t-Stat     ┃ Null Val ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩ │
│ │ ev                             │ -2.9469584     │  0.88728791    │ -3.321     │  0       │ │
│ │ hiperf                         │  0.17956774    │  0.13600659    │  1.32      │  0       │ │
│ │ hybrid                         │  0.58573044    │  0.21114577    │  2.774     │  0       │ │
│ │ max_range                      │  0.7285195     │  0.39282927    │  1.855     │  0       │ │
│ │ medhiperf                      │  0.79068441    │  0.24306823    │  3.253     │  0       │ │
│ │ opcost_scaled                  │ -0.23593174    │  0.080447167   │ -2.933     │  0       │ │
│ │ price_scaled                   │ -0.68940335    │  0.12854235    │ -5.363     │  0       │ │
│ │ s_ev                           │  2.6336908     │  0.89315379    │  2.949     │  0       │ │
│ │ s_hiperf                       │  0.86662755    │  0.75971091    │  1.141     │  0       │ │
│ │ s_hybrid                       │ -1.2469929     │  0.59955537    │ -2.08      │  0       │ │
│ │ s_max_range                    │  1.0485946     │  0.63013548    │  1.664     │  0       │ │
│ │ s_medhiperf                    │  1.637088      │  0.69212246    │  2.365     │  0       │ │
│ │ s_opcost_scaled                │  0.59556314    │  0.22095694    │  2.695     │  0       │ │
│ └────────────────────────────────┴────────────────┴────────────────┴────────────┴──────────┘ │
└──────────────────────────────────────────────────────────────────────────────────────────────┘

      message: Optimization terminated successfully
      success: True
       status: 0
            x: [-2.947e+00  1.796e-01 ...  1.637e+00  5.956e-01]
          nit: 32
          jac: [ 8.348e-03 -2.515e-02 ... -9.024e-05  2.711e-02]
         nfev: 81
         njev: 32
      n_cases: 1484
 total_weight: 1484.0
      logloss: 0.9336688512097793
      loglike: -1385.5645751953125
       stderr: [ 8.873e-01  1.360e-01 ...  6.921e-01  2.210e-01]

In [7]:
mixed = simple.copy()
for k in ["opcost_scaled", "max_range", "ev", "hybrid", "hiperf", "medhiperf"]:
    mixed.mixtures.append(lx.mixtures.Normal(k, f"s_{k}"))
mixed.n_draws = 200
mixed.seed = 150
mixed.maximize_loglike(stderr=True, options={"ftol": 1e-9})

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Larch Model Dashboard                                                                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ computing standard errors...                                                          │
│ Log Likelihood Current =      -1399.193359 Best =      -1399.193359                   │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓ │
│ ┃ Parameter                      ┃ Value          ┃ Gradient       ┃ Best           ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩ │
│ │ ev                             │ -1.3941456     │  0.010291226   │ -1.3941457     │ │
│ │ hiperf                         │  0.10999267    │ -0.034587689   │  0.10999294    │ │
│ │ hybrid                         │  0.3553575     │  0.064832807   │  0.35535699    │ │
│ │ max_range                      │  0.47840461    │ -0.030103367   │  0.47840485    │ │
│ │ medhiperf                      │  0.38431965    │ -0.040418498   │  0.38431996    │ │
│ │ opcost_scaled                  │ -0.12877089    │ -0.16830136    │ -0.12876958    │ │
│ │ price_scaled                   │ -0.41683012    │  0.25158107    │ -0.41683209    │ │
│ │ s_ev                           │  0.0010004772  │  0.061079621   │  0.001         │ │
│ │ s_hiperf                       │  0.00099990362 │ -0.01234287    │  0.001         │ │
│ │ s_hybrid                       │  0.00099997059 │ -0.0037710965  │  0.001         │ │
│ │ s_max_range                    │  0.0010005722  │  0.073238216   │  0.001         │ │
│ │ s_medhiperf                    │  0.00099981846 │ -0.023243606   │  0.001         │ │
│ │ s_opcost_scaled                │  0.00099974968 │ -0.032046795   │  0.001         │ │
│ └────────────────────────────────┴────────────────┴────────────────┴────────────────┘ │
└───────────────────────────────────────────────────────────────────────────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Larch Model Dashboard                                                                        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ optimization complete                                                                        │
│ Log Likelihood Current =      -1399.193359 Best =      -1399.193359                          │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓ │
│ ┃ Parameter                      ┃ Estimate       ┃ Std. Error     ┃ t-Stat     ┃ Null Val ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩ │
│ │ ev                             │ -1.3941456     │  0.27667764    │ -5.039     │  0       │ │
│ │ hiperf                         │  0.10999267    │  0.083826728   │  1.312     │  0       │ │
│ │ hybrid                         │  0.3553575     │  0.1218166     │  2.917     │  0       │ │
│ │ max_range                      │  0.47840461    │  0.17650138    │  2.71      │  0       │ │
│ │ medhiperf                      │  0.38431965    │  0.085456513   │  4.497     │  0       │ │
│ │ opcost_scaled                  │ -0.12877089    │  0.035309725   │ -3.647     │  0       │ │
│ │ price_scaled                   │ -0.41683012    │  0.033163123   │ -12.57     │  0       │ │
│ │ s_ev                           │  0.0010004772  │  nan           │  nan       │  0       │ │
│ │ s_hiperf                       │  0.00099990362 │  0.59385979    │  0.001684  │  0       │ │
│ │ s_hybrid                       │  0.00099997059 │  nan           │  nan       │  0       │ │
│ │ s_max_range                    │  0.0010005722  │  nan           │  nan       │  0       │ │
│ │ s_medhiperf                    │  0.00099981846 │  nan           │  nan       │  0       │ │
│ │ s_opcost_scaled                │  0.00099974968 │  nan           │  nan       │  0       │ │
│ └────────────────────────────────┴────────────────┴────────────────┴────────────┴──────────┘ │
└──────────────────────────────────────────────────────────────────────────────────────────────┘

      message: Optimization terminated successfully
      success: True
       status: 0
            x: [-1.394e+00  1.100e-01 ...  9.998e-04  9.997e-04]
          nit: 1
          jac: [ 1.029e-02 -3.459e-02 ... -2.324e-02 -3.205e-02]
         nfev: 12
         njev: 1
      n_cases: 1484
 total_weight: 1484.0
      logloss: 0.9428526680424528
      loglike: -1399.193359375
       stderr: [ 2.767e-01  8.383e-02 ...        nan        nan]

In [6]:
mixed = simple.copy()
for k in ["opcost_scaled", "max_range", "ev", "hybrid", "hiperf", "medhiperf"]:
    mixed.mixtures.append(lx.mixtures.Normal(k, f"s_{k}"))
mixed.n_draws = 200
mixed.seed = 200
mixed.maximize_loglike(stderr=True, options={"ftol": 1e-9})

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Larch Model Dashboard                                                                        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ optimization complete                                                                        │
│ Log Likelihood Current =      -1385.371094 Best =      -1385.371094                          │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓ │
│ ┃ Parameter                      ┃ Estimate       ┃ Std. Error     ┃ t-Stat     ┃ Null Val ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩ │
│ │ ev                             │ -3.2733885     │  0.9533155     │ -3.434     │  0       │ │
│ │ hiperf                         │  0.18456029    │  0.14306249    │  1.29      │  0       │ │
│ │ hybrid                         │  0.59270715    │  0.21769936    │  2.723     │  0       │ │
│ │ max_range                      │  0.86408334    │  0.44415012    │  1.945     │  0       │ │
│ │ medhiperf                      │  0.81021977    │  0.25323296    │  3.2       │  0       │ │
│ │ opcost_scaled                  │ -0.24844257    │  0.086484484   │ -2.873     │  0       │ │
│ │ price_scaled                   │ -0.71162996    │  0.13319589    │ -5.343     │  0       │ │
│ │ s_ev                           │  3.0144157     │  0.96091157    │  3.137     │  0       │ │
│ │ s_hiperf                       │ -1.0616638     │  0.63454223    │ -1.673     │  0       │ │
│ │ s_hybrid                       │  1.2956302     │  0.7127862     │  1.818     │  0       │ │
│ │ s_max_range                    │  0.873276      │  1.1766349     │  0.7422    │  0       │ │
│ │ s_medhiperf                    │ -1.7542238     │  0.71651083    │ -2.448     │  0       │ │
│ │ s_opcost_scaled                │  0.61040227    │  0.22911571    │  2.664     │  0       │ │
│ └────────────────────────────────┴────────────────┴────────────────┴────────────┴──────────┘ │
└──────────────────────────────────────────────────────────────────────────────────────────────┘

      message: Optimization terminated successfully
      success: True
       status: 0
            x: [-3.273e+00  1.846e-01 ... -1.754e+00  6.104e-01]
          nit: 33
          jac: [ 2.938e-04 -3.382e-02 ... -1.211e-03  1.809e-02]
         nfev: 79
         njev: 33
      n_cases: 1484
 total_weight: 1484.0
      logloss: 0.9335384728773585
      loglike: -1385.37109375
       stderr: [ 9.533e-01  1.431e-01 ...  7.165e-01  2.291e-01]

*The best loglikehood values for different seeds vary slightly; seed change does not change with the best LL result; the standard errors fluctuate widely across seeds, which can alter the statistical significance of parameters.*


## Other Specifications

Now let's return to specification issues. We have seen that the use of
normal distributions creates unrealistic results for coefficients that
should be the same sign for all people. Change the distributions for
the operating cost, range, and the performance, to be `lx.mixtures.LogNormal`
instead of Normal. Also, this is important: the lognormal distribution
has support only on the positive side of zero. So, if you want to use
a lognormal distribution for an undesirable attribute (for which all
people have negative coefficients), then you need to use
`lx.mixtures.NegLogNormal` instead.

- Does this model fit the data better or worse than the model with
  normal distributions, based on the log-likelihood value?

- What are the estimated mean and standard deviation of the willingness
  to pay for operating cost reductions? How do they compare to those
  from the model with normal distributions?

- Now allow the price coefficient to have a lognormal coefficient. What
  is the estimated distribution for willingness to pay for operating cost
  reductions?

- Try other specification and find the model that you think is best.

In [8]:
mixed = simple.copy()
for k in ["opcost_scaled", "max_range", "ev", "hybrid", "hiperf", "medhiperf"]:
    mixed.mixtures.append(lx.mixtures.LogNormal(k, f"s_{k}"))
mixed.n_draws = 200
mixed.seed = 42
mixed.maximize_loglike(stderr=True, options={"ftol": 1e-9})

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Larch Model Dashboard                                                                        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ optimization complete                                                                        │
│ Log Likelihood Current =      -1415.015381 Best =      -1415.015381                          │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓ │
│ ┃ Parameter                      ┃ Estimate       ┃ Std. Error     ┃ t-Stat     ┃ Null Val ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩ │
│ │ ev                             │ -9.7980352     │  23.846424     │ -0.4109    │  0       │ │
│ │ hiperf                         │ -2.1457275     │  0.92865109    │ -2.311     │  0       │ │
│ │ hybrid                         │ -0.34846463    │  0.26715076    │ -1.304     │  0       │ │
│ │ max_range                      │ -8.4604425     │  15.292724     │ -0.5532    │  0       │ │
│ │ medhiperf                      │ -4.1490723     │  6.031354      │ -0.6879    │  0       │ │
│ │ opcost_scaled                  │ -6.2816184     │  8.5331631     │ -0.7361    │  0       │ │
│ │ price_scaled                   │ -0.53341024    │  0.039211188   │ -13.6      │  0       │ │
│ │ s_ev                           │ -0.099786823   │  23.984545     │ -0.00416   │  0       │ │
│ │ s_hiperf                       │ -0.12868356    │  3.7709603     │ -0.03412   │  0       │ │
│ │ s_hybrid                       │  0.77935232    │  0.46650729    │  1.671     │  0       │ │
│ │ s_max_range                    │  0.9816499     │  11.822477     │  0.08303   │  0       │ │
│ │ s_medhiperf                    │  4.1751094     │  5.9704537     │  0.6993    │  0       │ │
│ │ s_opcost_scaled                │ -0.57650328    │  12.307394     │ -0.04684   │  0       │ │
│ └────────────────────────────────┴────────────────┴────────────────┴────────────┴──────────┘ │
└──────────────────────────────────────────────────────────────────────────────────────────────┘

      message: Optimization terminated successfully
      success: True
       status: 0
            x: [-9.798e+00 -2.146e+00 ...  4.175e+00 -5.765e-01]
          nit: 68
          jac: [-1.773e-03 -5.931e-03 ... -2.644e-02  6.439e-03]
         nfev: 112
         njev: 68
      n_cases: 1484
 total_weight: 1484.0
      logloss: 0.9535144075871799
      loglike: -1415.015380859375
       stderr: [ 2.385e+01  9.287e-01 ...  5.970e+00  1.231e+01]